In [1]:
from scipy.spatial import distance 
import pandas as pd
import numpy as np


In [2]:
 # Dans cette section, on se concentrera sur l'impact de l'augmentation de la valeur K (de nombre plus proche voisin que le
#    modele utilise pour faire de la prédiction)

In [3]:
# Importation de la base de données
#train_df = pd.read_csv("paris_airbnb.csv")
#test_df = pd.read_csv("paris_airbnb.csv")
paris_listings = pd.read_csv("paris_airbnb.csv")

In [4]:
paris_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    5000 non-null   object 
 1   host_acceptance_rate  0 non-null      float64
 2   host_listings_count   7999 non-null   float64
 3   latitude              8000 non-null   float64
 4   longitude             8000 non-null   float64
 5   city                  7997 non-null   object 
 6   zipcode               7930 non-null   object 
 7   state                 7977 non-null   object 
 8   accommodates          8000 non-null   int64  
 9   room_type             8000 non-null   object 
 10  bedrooms              7976 non-null   float64
 11  bathrooms             7942 non-null   float64
 12  beds                  7986 non-null   float64
 13  price                 8000 non-null   object 
 14  cleaning_fee          6250 non-null   object 
 15  security_deposit     

In [5]:
paris_listings["price"]=paris_listings["price"].apply(lambda x: x.replace(",", ""))
paris_listings["price"]=paris_listings["price"].apply(lambda x: x.replace("$", ""))
paris_listings["price"]=paris_listings["price"].apply(lambda x: float(x))

*Dans cette section on ne souhaite pas garder les colonnes qui ne contiennent pas de valeurs non numériques, et des 
valeurs numériques mais non ordinales*

### Les colonnes de valeurs non numériques:

**room_type**,
**city**,
**state**,

### Colonnes de valeurs non ordinanles:

**latitude**,
**longitude**,
**zipcode**

### Colonnes ne décrivant pas de logement:

**host_reponse_rate**,
**host_acceptance_rate**,
**host_listings_count**,

**Suppression des caractéristiques, les 9 colonnes citées ci-dessus**

In [6]:
drop_columns = ['room_type','city', 'state', 'latitude', 'longitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate','host_listings_count']

In [7]:
paris_listings = paris_listings.drop(drop_columns, axis=1)
print(paris_listings.isnull().sum())

accommodates            0
bedrooms               24
bathrooms              58
beds                   14
price                   0
cleaning_fee         1750
security_deposit     1680
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64


#### Gestion des valeurs manquantes. On peut supprimer les valeurs manquantes de ces trois colonnes car ces valeurs est faible

bedrooms 24;
bathrooms 58;
beds 14;
cleaning_fee    1750;
security_deposit    1680;

In [8]:
paris_listings = paris_listings.drop(['security_deposit', 'cleaning_fee'], axis=1) # ça supp les colonnes
paris_listings = paris_listings.dropna(axis=0) # ça va supp toutes les lignes qui contiennet des valeurs manquantes
print(paris_listings.isnull().sum())

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64


In [9]:
# Ces 8 colonnes vont nous être utiles pour notre modele maintenant. 

In [10]:
paris_listings.head(6)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,2,0.0,1.0,0.0,75.0,3,15,8
1,2,0.0,1.0,1.0,115.0,1,90,173
2,4,2.0,1.0,2.0,115.0,10,23,243
3,3,1.0,1.0,1.0,90.0,3,365,1
4,2,1.0,1.0,1.0,75.0,180,365,0
5,2,1.0,1.0,1.0,75.0,5,120,17


### On va normaliser toutes les colonnes pour éviter l'effet demesurer de la distance euclidienne. Ramener toutes les colonnes 
à la même echelle pour avoir une moyenne de 0 et un écartype de 1. C'est qui est la distribution normale standard.
Normaliser les valeurs de chanque colonnes en suivant cette distribution standard preserve la distribution de chaque colonne
par rapport à leur echelle. Pour cela on a besoin pour chaque valeur, la soustraire à la moyenne de chaque colonne et diviser
chaque par la déviation standard de la colonne. La formule mathematique qui décrit cette transformation est la suivante:
x = (x-£)/sigma
x: est une valeur spécifique; mi: est la moyenne de la valeur de toutes les colonnes et sigma c'est l'écart type de toute
        la colonne
.

In [11]:
# 1) soustraire chaque valeur dans la colonne par moyenne
first_transformation = paris_listings['maximum_nights'] - paris_listings['maximum_nights'].mean()

# 2) Diviser chaque valeur dans la colonne par l'ecart type
normalized_col = first_transformation / first_transformation.std()

In [12]:
normalized_listings = (paris_listings - paris_listings.mean()) / paris_listings.std()
normalized_listings['price'] = paris_listings['price']
print('Voilà nos valeurs normalisées selon la distribution normale standard pour les 3 1ère lignes')
print()
print(normalized_listings.head(3))
print()
print('On remarque sur la colonne price, on a nos valeurs non normalisées')

Voilà nos valeurs normalisées selon la distribution normale standard pour les 3 1ère lignes

   accommodates  bedrooms  bathrooms     beds  price  minimum_nights  \
0     -0.767566 -1.486372  -0.293537 -1.49583   75.0       -0.156795   
1     -0.767566 -1.486372  -0.293537 -0.64526  115.0       -0.212407   
2      0.503868  0.892605  -0.293537  0.20531  115.0        0.037847   

   maximum_nights  number_of_reviews  
0       -0.979751          -0.535593  
1       -0.841737           1.852943  
2       -0.965029           2.866261  

On remarque sur la colonne price, on a nos valeurs non normalisées


#### Distance euclidienne pour le cas multivarié

In [13]:
# bibliothèque de la distance euclidienne
from scipy.spatial import distance 

In [14]:
# exemple de calcul de la distance euclidienne
l1 = [0.503868, -0.296884]
l2 = [-0.131849, 0.892605]

distance = distance.euclidean(l1,l2)
distance

1.3487105639128063

In [15]:
# Calcul de la distance euclidienne de la colonne 'accommodated et bedrooms'

firs_listing = normalized_listings.iloc[0][["accommodates", "bedrooms"]]
second_listing = normalized_listings.iloc[4][["accommodates", "bedrooms"]]
distance_accommod_bedroom = distance.euclidean(firs_listing,second_listing)
print(distance_accommod_bedroom)

AttributeError: 'float' object has no attribute 'euclidean'

## Utilisation de la bibliothèque Scikit-Learn pour KNN Multivarié

****Classe KNeighborsRegressor :**** La regression de K plus proches voisins de regression


Syntaxe:
- **n_neighborsint, default=5 :**(Nombre de voisins à utiliser par défaut pour kneighborsles requêtes, par defaut K = 5)
- **algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}, default=’auto’**
- **p entier, par défaut=2**

In [16]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(algorithm='brut')

*Adapter notre modèle pour faire des prédictions*

In [17]:
# 1)Diviser le dataset en deux partie
train_df = normalized_listings.iloc[0:6000]
test_df = normalized_listings.iloc[6000:]

# 2) Seule l'objet de type matrice contenu les 2 colonnes de notre set de training qui nous intéressent
train_features = train_df[['accommodates', 'bedrooms']]

# 3) Objet de type liste contenant juste la colonne cible 'price'
train_target = train_def['price']

# 4) Passer tous ces objets dans la méthode fit
knn.fit(train_features, train_target)

# 5) Utilisation de la méthode prdict(). Cette méthode prend qu'un seul parametre. Un objet matrice contenant les caractéristique
  #du set de test
    
predictions = knn.predict(test_df[['accommodates', 'bedrooms']])

NameError: name 'train_def' is not defined